<a href="https://colab.research.google.com/github/DevNicque/pos-datascience/blob/main/C%C3%B3pia_de_equipe_4_cota_parlamentar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aquisição, pré-processamento e exploração de dados - Equipe 4

**Alunos**:

* Andrea Monicque dos Santos Silva (amdss.cid20@uea.edu.br)
* Francisco Marcelo Mendes Damasceno (fmmd.cid20@uea.edu.br)
* Marcos Wenneton Araújo (mwvda.cid20@uea.edu.br)
* Lucas Pereira Reis (lpr.cid20@uea.edu.br)


**Projeto**:

1. O trabalho será feito por equipe.
2. Vocês vão definir questões que sejam interessantes para um negócio e que para ser respondidas precisem da Ciência dos Dados. O negócio pode ser qualquer área de atuação.  
3. Coleta de Dados. Fazer a coleta de dados das fontes de dados disponíveis (pode ser uma fonte ou várias). Avaliando se têm dados para responder as questões identificadas no ponto 2. Deve avaliar a qualidade dos dados a partir das dimensões de qualidade.
4. Aplicar métodos de pré-processamento de dados estudados, explicando para que é cada método (ao aplicar os métodos poderão ter algumas respostas). Obter um dataset de qualidade pronto para aplicar método de ML. 
5. Explorar seus dados, para responder algumas das questões definidas no ponto 2. Outras questões poderão ser só respondidas depois de usar métodos de ML.
6. Todos os documentos do projeto deveram ser postados no GitHub, e deverá ser feita uma apresentação na última aula da disciplina.
7. Seu projeto vai ser avaliado por outros Equipes e a nota final do projeto vai levar em conta as avaliações feita pelos outros equipes e a avaliação feita pelo professor. 

###**Questões**

1. Quais deputados tiveram os maiores gastos anuais no período entre 2015 e 2020?
2. Em relação aos anos anteriores, o gasto com cota parlamentar aumentou ou diminuiu durante a pandemia?
3. Em quais áreas e/ou serviços foram realizados os maiores gastos?
4. Em relação aos partidos políticos, quais gastaram mais em relação aos outros?

###**Coleta de Dados**

In [2]:
!pip install unidecode

     |████████████████████████████████| 245kB 8.5MB/s 


In [3]:
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objects as go
pd.options.display.float_format = '{:.2f}'.format

In [106]:
# Carregando os datasets de acordo com um intervalo de anos
df_base_quota = pd.DataFrame({})
for year in range(2015, 2022):
  df = pd.read_csv(f'https://www.camara.leg.br/cotas/Ano-{year}.csv.zip', compression='zip', error_bad_lines=False, sep=";", dtype={'cpf':'object','ideCadastro':'object','nuCarteiraParlamentar':'object',})
  df_base_quota = df_base_quota.append(df)

In [108]:
# Dataset auxiliar com os limites mensais coletados de:
dflimite = pd.read_csv('http://underlabs.org/datasets/limite_mensal_cota_por_estado.csv')

**Base Quota Parlamentar**

In [110]:
df_base_quota.head()

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,numEspecificacaoSubCota,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,txtNumero,indTipoDocumento,datEmissao,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,105348,0,2015-06-12T00:00:00,500.00,0.00,500.00,6,2015,0,NaN,NaN,1212156,nan,nan,2865,5745686,https://www.camara.leg.br/cota-parlamentar/doc...
1,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,107516,0,2015-07-10T00:00:00,500.00,0.00,500.00,7,2015,0,NaN,NaN,1212157,nan,nan,2865,5745690,https://www.camara.leg.br/cota-parlamentar/doc...
2,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,109281,0,2015-08-01T00:00:00,500.00,0.00,500.00,8,2015,0,NaN,NaN,1218516,nan,nan,2865,5766095,https://www.camara.leg.br/cota-parlamentar/doc...
3,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,111330,0,2015-08-28T00:00:00,500.00,0.00,500.00,9,2015,0,NaN,NaN,1225794,nan,nan,2865,5790638,https://www.camara.leg.br/cota-parlamentar/doc...
4,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,113548,0,2015-09-29T00:00:00,333.00,0.00,333.00,10,2015,0,NaN,NaN,1235281,nan,nan,2865,5821986,https://www.camara.leg.br/cota-parlamentar/doc...


**Base Limite de gasto por estado**

In [111]:
dflimite.head()

,uf,deputados,limite
0,AC,8,44632.46
1,AL,9,40944.10
2,AM,8,43570.12
3,AP,8,43374.78
4,BA,39,39010.85


###**Redução e Limpeza de Dados**


In [115]:
df_quota = df_base_quota.copy()

**verificando duplicatas**

In [93]:
df_dup2 = df_quota[df_quota.duplicated()]

In [94]:
df_dup2.shape[0]

0

**Verificação dos dados**

In [116]:
df_quota.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891770 entries, 0 to 14077
Data columns (total 31 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   txNomeParlamentar          object 
 1   cpf                        object 
 2   ideCadastro                object 
 3   nuCarteiraParlamentar      object 
 4   nuLegislatura              int64  
 5   sgUF                       object 
 6   sgPartido                  object 
 7   codLegislatura             int64  
 8   numSubCota                 int64  
 9   txtDescricao               object 
 10  numEspecificacaoSubCota    int64  
 11  txtDescricaoEspecificacao  object 
 12  txtFornecedor              object 
 13  txtCNPJCPF                 object 
 14  txtNumero                  object 
 15  indTipoDocumento           int64  
 16  datEmissao                 object 
 17  vlrDocumento               float64
 18  vlrGlosa                   float64
 19  vlrLiquido                 float64
 20  numM

**Removendo informações inicialmente desnecessárias para a análise** 

In [117]:
df_quota = df_quota.drop(columns=['txtPassageiro','txtTrecho','urlDocumento'])

**Renomeando os campos**

In [118]:
df_quota.columns = ['parlamentar', 'cpf', 'id_cadastro', 'num_carteira','ano_legislatura','uf','partido','cod_legislatura','cod_despesa','despesa','cod_tipo_despesa','tipo_despesa','fornecedor','fornec_CNPJ_CPF','num_doc','tipo_documento','data_emissao','valor_doc','valor_glosa','valor_liquido','mes','ano','parcelas','num_lote','num_ressarciamento','valor_restituicao','id_deputado','id_documento']

In [119]:
df_quota.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1891770 entries, 0 to 14077
Data columns (total 28 columns):
 #   Column              Dtype  
---  ------              -----  
 0   parlamentar         object 
 1   cpf                 object 
 2   id_cadastro         object 
 3   num_carteira        object 
 4   ano_legislatura     int64  
 5   uf                  object 
 6   partido             object 
 7   cod_legislatura     int64  
 8   cod_despesa         int64  
 9   despesa             object 
 10  cod_tipo_despesa    int64  
 11  tipo_despesa        object 
 12  fornecedor          object 
 13  fornec_CNPJ_CPF     object 
 14  num_doc             object 
 15  tipo_documento      int64  
 16  data_emissao        object 
 17  valor_doc           float64
 18  valor_glosa         float64
 19  valor_liquido       float64
 20  mes                 int64  
 21  ano                 int64  
 22  parcelas            int64  
 23  num_lote            int64  
 24  num_ressarciamento  float6

**Tratando sensibilidade de strings**

In [120]:
df_quota.head()

,parlamentar,cpf,id_cadastro,num_carteira,ano_legislatura,uf,partido,cod_legislatura,cod_despesa,despesa,cod_tipo_despesa,tipo_despesa,fornecedor,fornec_CNPJ_CPF,num_doc,tipo_documento,data_emissao,valor_doc,valor_glosa,valor_liquido,mes,ano,parcelas,num_lote,num_ressarciamento,valor_restituicao,id_deputado,id_documento
0,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,105348,0,2015-06-12T00:00:00,500.00,0.00,500.00,6,2015,0,1212156,nan,nan,2865,5745686
1,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,107516,0,2015-07-10T00:00:00,500.00,0.00,500.00,7,2015,0,1212157,nan,nan,2865,5745690
2,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,109281,0,2015-08-01T00:00:00,500.00,0.00,500.00,8,2015,0,1218516,nan,nan,2865,5766095
3,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,111330,0,2015-08-28T00:00:00,500.00,0.00,500.00,9,2015,0,1225794,nan,nan,2865,5790638
4,LIDERANÇA DO PARTIDO REPUBLICANO DA ORDEM SOCIAL,NaN,NaN,NaN,2015,NaN,NaN,55,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE ...,0,NaN,COMERCIAL DE ALIMENTOS GRANO LTDA,062.784.330/0019-0,113548,0,2015-09-29T00:00:00,333.00,0.00,333.00,10,2015,0,1235281,nan,nan,2865,5821986


In [121]:
df_quota.parlamentar = df_quota.parlamentar.str.upper()
df_quota.uf = df_quota.uf.str.upper()
df_quota.partido = df_quota.partido.str.upper()
df_quota.despesa = df_quota.despesa.str.upper()
df_quota.tipo_despesa = df_quota.tipo_despesa.str.upper()
df_quota.fornecedor = df_quota.fornecedor.str.upper()

In [42]:
# Alterando a descrição de caixa alta para apenas a primeira letra em caixa alta
#df_quota.despesa = df_quota.despesa.apply(lambda x: x.title())

In [122]:
#Removendo Acentos
for k in ['parlamentar','despesa']:
    df_quota[k]= [unidecode(x) for x in df_quota[k]]

In [102]:
df_quota.despesa.value_counts()

PASSAGEM AEREA - RPA                                         527670
COMBUSTIVEIS E LUBRIFICANTES.                                389427
TELEFONIA                                                    253205
SERVICO DE TAXI, PEDAGIO E ESTACIONAMENTO                    154485
MANUTENCAO DE ESCRITORIO DE APOIO A ATIVIDADE PARLAMENTAR    135802
SERVICOS POSTAIS                                             127632
FORNECIMENTO DE ALIMENTACAO DO PARLAMENTAR                   117411
DIVULGACAO DA ATIVIDADE PARLAMENTAR.                          66237
LOCACAO OU FRETAMENTO DE VEICULOS AUTOMOTORES                 37294
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.        35647
CONSULTORIAS, PESQUISAS E TRABALHOS TECNICOS.                 15878
PASSAGEM AEREA - REEMBOLSO                                    14106
PASSAGENS TERRESTRES, MARITIMAS OU FLUVIAIS                    5741
SERVICO DE SEGURANCA PRESTADO POR EMPRESA ESPECIALIZADA.       4757
ASSINATURA DE PUBLICACOES                       

In [103]:
df_quota.tipo_despesa.value_counts()

VEÍCULOS AUTOMOTORES    386721
AERONAVES                 1651
SEM ESPECIFICAÇÕES         887
EMBARCAÇÕES                168
Name: tipo_despesa, dtype: int64

**Tratando dados faltantes**

In [124]:
df_quota.isna().sum()

parlamentar                 0
cpf                      4794
id_cadastro              4628
num_carteira             4628
ano_legislatura             0
uf                       4628
partido                  4628
cod_legislatura             0
cod_despesa                 0
despesa                     0
cod_tipo_despesa            0
tipo_despesa          1502343
fornecedor                 33
fornec_CNPJ_CPF            38
num_doc                     0
tipo_documento              0
data_emissao           121249
valor_doc                   0
valor_glosa                 0
valor_liquido               0
mes                         0
ano                         0
parcelas                    0
num_lote                    0
num_ressarciamento    1078597
valor_restituicao     1890854
id_deputado                 0
id_documento                0
dtype: int64

In [125]:
df_quota.update(df_quota['cpf'].fillna('00000000000'))          #nem todos registros são de pessoas físicas, alguns são lideranças partidárias
df_quota.update(df_quota['id_cadastro'].fillna('000000'))
df_quota.update(df_quota['num_carteira'].fillna('000'))
df_quota.update(df_quota['ano_legislatura'].fillna(''))
df_quota.update(df_quota['uf'].fillna('NAO ESPECIFICADO'))
df_quota.update(df_quota['partido'].fillna('NAO ESPECIFICADO'))
df_quota.update(df_quota['tipo_despesa'].fillna('SEM ESPECIFICAÇÕES'))
df_quota.update(df_quota['fornec_CNPJ_CPF'].fillna(''))
df_quota.update(df_quota['data_emissao'].fillna('0000-00-00T00:00:00'))
df_quota.update(df_quota['num_ressarciamento'].fillna(0))
df_quota.update(df_quota['valor_restituicao'].fillna(0))


In [127]:
#há tipos de despesas como telefonia que não gera um documento, com isso colocando como zero
df_quota.id_documento.value_counts() 

0          285406
5716890         4
5728480         3
6066113         3
5843683         3
            ...  
6858685         1
6864830         1
6862783         1
7032768         1
7071874         1
Name: id_documento, Length: 1604793, dtype: int64

**Tratando data**

In [ ]:
df_quota.data_emissao = pd.to_datetime(df_quota.data_emissao).apply(lambda x: x.date())

**Verificando dados duplicados**

In [128]:
df_dup = df_quota[df_quota.duplicated()]

In [129]:
df_dup.shape[0]

0

####**Análise de Correlaçao**

#####**Tipo de documuento x Gasto**

tipo de documento: 0-NF, 1-Recibo, 2-Despesas no exterior

Analisar se o tipo de documento tem relação com o valor do gasto. Será se uma compra ser feita no exterior influencia em um valor alto de produto mais do que uam compra com nota fiscal?

In [143]:
df_cor2 = df_quota[['tipo_documento','valor_liquido']]

In [ ]:

trace2 = go.Scatter(x = df_cor2['valor_liquido'],
                   y = df_cor2['tipo_documento'],
                   mode = 'markers')
data2 = [trace2]
py.iplot(data2)

In [144]:
df_cor2.cov()

,tipo_documento,valor_liquido
tipo_documento,1.29,-16.65
valor_liquido,-16.65,4816534.83


In [145]:
df_cor2.corr(method='pearson')

,tipo_documento,valor_liquido
tipo_documento,1.00,-0.01
valor_liquido,-0.01,1.00


In [146]:
df_cor2.corr(method='spearman')

,tipo_documento,valor_liquido
tipo_documento,1.00,-0.18
valor_liquido,-0.18,1.00


#####**Despesa x gastos**

In [1]:
df_cor3 = df_quota[['tipo_despesa','valor_liquido']]

NameError: ignored

In [152]:
df_cor3.cov()

,cod_despesa,valor_liquido
cod_despesa,192594.04,-41346.92
valor_liquido,-41346.92,4816534.83


In [153]:
df_cor3.corr(method='pearson')

,cod_despesa,valor_liquido
cod_despesa,1.00,-0.04
valor_liquido,-0.04,1.00


In [154]:
df_cor3.corr(method='spearman')

,cod_despesa,valor_liquido
cod_despesa,1.00,-0.02
valor_liquido,-0.02,1.00


###**Transformação de Dados**

Etapa onde os dados são convertidos ou consolidados em um novo formato, também é criado novos atributos que agregam os existentes

In [ ]:
dflimite['limite_anual'] = dflimite.limite * dflimite.deputados * 12

###**Insights**

#### 2. Em relação aos anos anteriores, o gasto com cota parlamentar aumentou ou diminuiu durante a pandemia? - Marcelo

##### Q2. Análise e exploração dos dados

In [ ]:
# DF Gastos por Estado
dfgpe = df_quota[df_quota.sguf.notna() == True][['numano', 'sguf', 'vlrliquido']].groupby(['numano', 'sguf'])['vlrliquido'].sum().reset_index()
dfgpe = pd.merge(dfgpe, dflimite, left_on='sguf', right_on='uf').drop(columns=['uf','deputados','limite'])
dfgpe['limite_usado'] = (dfgpe.vlrliquido * 100)/dfgpe.limite_anual

In [ ]:
# DF Gastos Totais Anuais
dfgta = df_quota[['numano', 'vlrliquido']].groupby(['numano'])['vlrliquido'].sum().reset_index()
dfgta['diferenca'] = (dfgta['vlrliquido'].pct_change() * 100)

In [ ]:
fig = px.bar(dfgta, x="numano", y="vlrliquido", title="[Q2.1] Evolução dos gastos totais com Corta Parlamentar de 2015 a 2020", width=640, height=400)
fig.show()

O gráfico [Q2.1] mostra que o uso da Cota para o Exercício da Atividade Parlamentar (CEAP) permanecia estável com leve alta até 2017, quando alguns estados começaram a reduzir seu uso. Essa tendência foi observada também nos demais estados nos anos subsequentes até o ano de 2020 quando houve uma forte queda.

In [ ]:
fig = px.line(dfgpe, x="numano", y="vlrliquido", color="sguf", line_group="sguf", hover_name="sguf", title="[Q2.2] Evolução dos gastos com Cota Parlamentar por Estado no período de 2015 a 2020", width=1280)
fig.show()

In [ ]:
df19 = dfgpe[dfgpe.numano == 2019]
df20 = dfgpe[dfgpe.numano == 2020]

fig = go.Figure(data=[
    go.Bar(name='2019', x=df19.sguf, y=df19.limite_usado),
    go.Bar(name='2020', x=df20.sguf, y=df20.limite_usado),
])
# Change the bar mode
fig.update_layout(barmode='group', title='[Q2.3] Contraste do Percentual de uso da Cota Parlamentar por estado entre os anos de 2019 e 2020',width=1280, height=400)
fig.show()

Nos gráficos [Q2.2] e [Q2.3] podemos observar que todos os estados seguiram a tendência de queda para o ano de 2020, alguns com maior intensidade que outros.

In [ ]:
fig = px.line(dfgta, x="numano", y="diferenca", title="[Q2.4] Diferena Percentual no uso da Cota Parlamentar em relação aos anos anteriores", labels={'diferenca':'Diferença Percentual', 'numano':'Ano'}, width=640, height=400)
fig.show()

##### Q2. Conclusão

Pode-se afirmar, com base nos dados e nos gráficos apresentados que houve uma **redução** atípica dos gastos de aproximadamente 23.86% no ano de **2020**, período em que a **pandemia** se estabeleceu no Brasil.

#### 3. Em quais áreas e/ou serviços foram realizados os maiores gastos? - Lucas

In [ ]:
df3 = df_quota.copy()
df3 = df3.rename({'txtdescricao': 'Descrição', 'vlrliquido': 'Valor Líquido'}, axis=1)

In [ ]:
df3_count = df3[['Descrição']].groupby(['Descrição']).size().reset_index(name="Quantidade").sort_values(by=['Quantidade']).reset_index().drop(['index'], axis=1)

px.bar(data_frame=df3_count, x="Descrição", y="Quantidade", barmode="group", title="Quantidade de vezes que cada tipo de despesa apareceu entre os anos de 2015 e 2021")

In [ ]:
max_quantity = df3_count['Quantidade'].max()

df3_count['Diferença (Razão)'] = df3_count.apply(lambda x: max_quantity / x['Quantidade'], axis=1)

df3_count

In [ ]:
df3_sum = df3[['Descrição', 'Valor Líquido']].groupby(['Descrição']).sum().sort_values(by=['Valor Líquido']).reset_index()

fig = px.pie(df3_sum, values='Valor Líquido', names='Descrição', title='Tipos de despesa que a cota foi gasta entre os anos de 2015 e 2021')
fig.show()

Durante o pedido em análise (2015 a 2021), "Passagem Aérea - Rpa" e "Combustíveis e Lubrificantes" foram as despesas que mais apareceram, mas eles não foram necessariamente as despesas que mais geraram gastos para o dinheiro público.
A despesa "Divulgação da Atividade Parlamentar" apresenta-se como o maior gasto com 25.3%, mesmo sendo apenas a 8ª despesa que mais apareceu nos dados. A maior despesa ("Passagem Aérea - Rpa") que apresenta uma quantidade 7,9 vezes maior encontra-se em segundo 21.4% dos gastos.


In [ ]:
df3_2020 = df3[df3.numano == 2020].copy()

In [ ]:
df3_2020_count = df3_2020[['Descrição']].groupby(['Descrição']).size().reset_index(name="Quantidade").sort_values(by=['Quantidade']).reset_index().drop(['index'], axis=1)

px.bar(data_frame=df3_2020_count, x="Descrição", y="Quantidade", barmode="group", title="Quantidade de vezes que cada tipo de despesa apareceu durante os gastos na pandemia")

In [ ]:
max_quantity = df3_2020_count['Quantidade'].max()

df3_2020_count['Diferença (Razão)'] = df3_2020_count.apply(lambda x: max_quantity / x['Quantidade'], axis=1)

df3_2020_count

In [ ]:
df3_2020_sum = df3_2020[['Descrição', 'Valor Líquido']].groupby(['Descrição']).sum().sort_values(by=['Valor Líquido']).reset_index()

fig = px.pie(df3_2020_sum, values='Valor Líquido', names='Descrição', title='Tipos de despesa que a cota foi gasta durante a pandemia')
fig.show()

É interessante observar que especificamente no ano da pandemia, "Passagem Aérea - Rpa" caiu para a 4ª despesa que mais apareceu, tendo um aumento para "Combustíveis E Lubrificantes" e "Manutenção De Escritório De Apoio à Atividade Palarmentar", este segundo provavelmente devido ao crescimento do home-office.

A "Divulgação Da Atividade Palarmentar" apareceu na 6ª colocação e mesmo sendo 3,3 vezes menor que a primeira, esta novamente foi o maior gasto com 33.7%.

#### 4. Em relação aos partidos políticos, quais gastaram mais em relação aos outros?

In [ ]:
df4 = df_quota.copy()

Remove dados de gastos sem partido:

In [ ]:
df4 = df4.dropna(subset=['sgpartido'])

In [ ]:
df4[df4['sgpartido'] == 'DEM']['vlrliquido'].sum()

89370984.83999999

In [ ]:
print("Proporção de gastos por deputado:")
df_partido_gasto = pd.DataFrame(columns=['Partido', 'Quantidade de Deputados', 'Valor gasto', 'Proporção de gasto'])
for partido in df4['sgpartido'].unique():
  df_partido = df4[df4['sgpartido'] == partido]
  gastos = df_partido['vlrliquido'].sum()
  qtde_deputados = len(df_partido['nudeputadoid'].unique())

  df_partido_gasto = pd.concat([df_partido_gasto, pd.DataFrame([[partido, qtde_deputados, gastos, gastos/qtde_deputados]],
                                             columns=['Partido', 'Quantidade de Deputados', 'Valor gasto', 'Proporção de gasto'])],
                                ignore_index=True)
  print('{}: {}'.format(partido, gastos/qtde_deputados))

Proporção de gastos por deputado:
DEM: 1241263.6783333332
PSB: 923117.7331428571
AVANTE: 1123764.447857143
PSC: 651109.0635714286
PSDB: 1085054.866736842
PMDB: 20415.745064935065
MDB: 1443322.364878049
PP: 1504042.7526250002
PP**: 23024.484047619047
SDD: 22315.222173913044
PODE: 1291870.3175862066
PT: 1100959.9657142856
PTB: 1144452.2157142856
PRB: 1219691.681
PPS: 952128.7006666665
PDT: 1116358.863846154
PSOL: 1144372.3961538463
PROS: 637088.61875
PV: 472356.89125000004
PR: 1149980.5836206896
SOLIDARIEDADE: 1358738.175
PSD: 935456.5878494624
PCdoB: 1116913.3865384613
PSL: 725046.296885246
REDE: 1107394.5566666666
PMN: 28491.713333333322
PATRI: 1644791.482
PHS: 1185680.2633333334
PTdoB: 20009.124999999996
PRP: 58677.405
PPL: 1871243.73
S.PART.: 14.18
PL: 657695.3157777778
PRTB: 167413.3
REPUBLICANOS: 700131.3174285714
CIDADANIA: 580519.99875
NOVO: 179049.01625
PATRIOTA: 740063.2216666667


In [ ]:
df_partido_gasto.head()

,Partido,Quantidade de Deputados,Valor gasto,Proporção de gasto
0,DEM,72,89370984.84,1241263.68
1,PSB,70,64618241.32,923117.73
2,AVANTE,14,15732702.27,1123764.45
3,PSC,28,18231053.78,651109.06
4,PSDB,95,103080212.34,1085054.87


In [ ]:
px.bar(data_frame=df_partido_gasto, x="Partido", y="Proporção de gasto", barmode="group", title="Proporção de gastos por partido")